In [43]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta
from sklearn.model_selection import train_test_split


print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Keras version: 		2.2.4
Scikit version: 	0.21.2
TensorFlow version: 	1.13.1


In [44]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence,remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [45]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')# Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
def review_to_wordlist( review, remove_stopwords=False ):
# Function to convert a document to a sequence of words,
# optionally removing stop words. Returns a list of words.
#
# 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
#
# 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
#
# 3. Convert words to lower case and split them
    words = review_text.lower().split()
#
# 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
#
# 5. Return a list of words
    return(words)


In [85]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
nltk.download('punkt')
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
# Function to split a review into parsed sentences. Returns a
# list of sentences, where each sentence is a list of words
#
# 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
#
# 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
# If a sentence is empty, skip it
        if len(raw_sentence) > 0:
# Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
#
# Return the list of sentences (each sentence is a list of words,
# so this returns a list of lists
    return sentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
import pandas as pd
path = '/content/gdrive/My Drive/data/labeledTrainData.tsv'

data1 = pd.read_csv(path, header=0, delimiter="\t", quoting=3)
data = data1['review']
target = data1['sentiment']

In [0]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

# create training and testing vars
train, test, ytrain, ytest = train_test_split(data, target, test_size=0.2)
#print X_train.shape, y_train.shape
#print X_test.shape, y_test.shape

In [0]:
xtrain = train.values.reshape(-1,1)

In [69]:
xtrain

array([['"What a load of Leftist Hollywood bilge. This movie glorifies mutiny as brave and noble if it be for pacifist principles. The fairytale ends with the pacifist character, played by Danzel Washington, actually getting promoted for his treason. What is it with these Hollywood tools? Is this still payback for McCarthyism?<br /><br />If I sound cynical it\'s because I am fed up with movies hawking a political agenda. The military brass in this movie are portrayed as, what else? Gung-ho war mongers. Sound familiar? Ever see a movie where the CIA or any government agency is not evil? Think about it. Yet again, Crimson Tide stresses the point. The Hackman character, U-boat captain Ramsey, comes across like a raving lunatic, until the very end when, of course he comes to his senses, does a complete 360, renounces his blood lust, suggests a promotion for the treasonous Ron Hunter, and repents by retiring from the service. A guy mutinies, takes command of your boat, puts the U.S at grave

In [63]:
'''
from google.colab import drive
import glob
import os
'''
'''
drive.mount('/content/gdrive/')
!ls "/content/gdrive/My Drive"
download_path = os.path.expanduser("/content/gdrive/My Drive")
try:
  os.makedirs(download_path)
except FileExistsError:
  pass
  
'''
'''
drive.mount('/content/gdrive/')


def read_files1(path):
    
    #drive.mount('/content/gdrive/')

    !mkdir drive
    !google-drive-ocamlfuse drive
    !ls "gdrive/My Drive/aclImdb"

    print(path)
    !ls path
    #path = glob.glob('/gdrive/aclImdb/train/pos/')
    print(path)
    documents = list()
    print("11111")
    i = 0
    for filename in os.listdir(path):
      with open('%s/%s' % (path, filename),encoding='utf-8') as f:
        print("2222")
        i = i+1
        print(i)
        doc = f.read()
        documents.append(doc)
        if i == 4:
          break
        
    if os.path.isfile(path):
        with open(path, encoding='iso-8859-1') as f:
            doc = f.readlines()
            for line in doc:
                documents.append(line)
    
    return documents
'''

'\ndrive.mount(\'/content/gdrive/\')\n\n\ndef read_files1(path):\n    \n    #drive.mount(\'/content/gdrive/\')\n\n    !mkdir drive\n    !google-drive-ocamlfuse drive\n    !ls "gdrive/My Drive/aclImdb"\n\n    print(path)\n    !ls path\n    #path = glob.glob(\'/gdrive/aclImdb/train/pos/\')\n    print(path)\n    documents = list()\n    print("11111")\n    i = 0\n    for filename in os.listdir(path):\n      with open(\'%s/%s\' % (path, filename),encoding=\'utf-8\') as f:\n        print("2222")\n        i = i+1\n        print(i)\n        doc = f.read()\n        documents.append(doc)\n        if i == 4:\n          break\n        \n    if os.path.isfile(path):\n        with open(path, encoding=\'iso-8859-1\') as f:\n            doc = f.readlines()\n            for line in doc:\n                documents.append(line)\n    \n    return documents\n'

In [64]:
'''
pos_train = read_files1('/content/gdrive/My Drive/aclImdb/train/pos')
neg_train = read_files1('/content/gdrive/My Drive/aclImdb/train/neg')
pos_test = read_files1('/content/gdrive/My Drive/aclImdb/test/pos')
neg_test = read_files1('/content/gdrive/My Drive/aclImdb/test/neg')


train1 = pos_train + neg_train
test1 = pos_test + neg_test

#docs = negative_docs + positive_docs
l_train = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_train))]
l_test = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_test))]
train = np.column_stack((train1,l_train))
test = np.column_stack((test1,l_test))
'''

"\npos_train = read_files1('/content/gdrive/My Drive/aclImdb/train/pos')\nneg_train = read_files1('/content/gdrive/My Drive/aclImdb/train/neg')\npos_test = read_files1('/content/gdrive/My Drive/aclImdb/test/pos')\nneg_test = read_files1('/content/gdrive/My Drive/aclImdb/test/neg')\n\n\ntrain1 = pos_train + neg_train\ntest1 = pos_test + neg_test\n\n#docs = negative_docs + positive_docs\nl_train = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_train))]\nl_test = [1 for _ in range(len(pos_train))] + [0 for _ in range(len(neg_test))]\ntrain = np.column_stack((train1,l_train))\ntest = np.column_stack((test1,l_test))\n"

In [86]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in data1["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [88]:
sentences[1]

['maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent']

Parsing sentences from training set


KeyError: ignored

In [89]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',    level=logging.INFO)

def myhashfxn(obj):
    return hash(obj) % (2 ** 32)

# Set values for various parameters
num_features = 400    # Word vector dimensionality                      
min_word_count = 60   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words


#**********************************************************



# Initialize and train the model (this will take some time)
from gensim.models import word2vec
#model = word2vec.Word2Vec(hashfxn=myhashfxn)

print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers,             size=num_features, min_count = min_word_count,             window = context, sample = downsampling,hashfxn=myhashfxn)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)


2019-06-06 02:15:44,719 : INFO : 'pattern' package not found; tag filters are not available for English
2019-06-06 02:15:44,729 : INFO : collecting all words and their counts
2019-06-06 02:15:44,730 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-06 02:15:44,786 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-06-06 02:15:44,845 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types
2019-06-06 02:15:44,901 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types


Training model...


2019-06-06 02:15:44,963 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-06-06 02:15:45,019 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-06-06 02:15:45,073 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-06-06 02:15:45,127 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-06-06 02:15:45,181 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-06-06 02:15:45,235 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-06-06 02:15:45,289 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-06-06 02:15:45,342 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 52081 word types
2019-06-06 02:15:45,397 : INFO : PROGRESS: at sentence #120000, processed 2668775 words, keepin

In [90]:
model.doesnt_match("man woman child kitchen".split())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [91]:

model.most_similar("man")



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('woman', 0.6654655933380127),
 ('doctor', 0.6339104175567627),
 ('boy', 0.6286064982414246),
 ('journalist', 0.594315767288208),
 ('businessman', 0.5905495285987854),
 ('lady', 0.5899538397789001),
 ('guy', 0.5748952627182007),
 ('soldier', 0.5712598562240601),
 ('scientist', 0.5709996223449707),
 ('priest', 0.5663358569145203)]

In [0]:
#Vector Averaging
import numpy as np # Make sure that numpy is imported
def makeFeatureVec(words, model, num_features):
# Function to average all of the word vectors in a given
# paragraph
#
# Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
#
    nwords = 0.
#
# Index2word is a list that contains the names of the words in
# the model's vocabulary. Convert it to a set, for speed
    index2word_set = set(model.wv.index2word)
#
# Loop over each word in the review and, if it is in the model's
# vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
#
# Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec



In [0]:
def getAvgFeatureVecs(reviews, model, num_features):
# Given a set of reviews (each one a list of words), calculate
# the average feature vector for each one and return a 2D numpy array
#
# Initialize a counter
    counter = 0
#
# Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
# Loop through the reviews
    for review in reviews:
        if counter%1000 == 0:
            print ("Review %d of %d" % (counter, len(reviews)))
        reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
#
# Increment the counter
        counter = counter + 1
    return reviewFeatureVecs




In [94]:
clean_train_reviews = []
for review in train1:
    clean_train_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))
    
trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )
print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test1:
    clean_test_reviews.append( review_to_wordlist( review,         remove_stopwords=True ))


# In[52]:

NameError: ignored